In [2]:
# import cv2
# import numpy as np


# def biggest_contour(contours):
#     biggest = np.array([])
#     max_area = 0
#     for i in contours:
#         area = cv2.contourArea(i)
#         if area > 1000:
#             peri = cv2.arcLength(i, True)
#             approx = cv2.approxPolyDP(i, 0.015 * peri, True)
#             if area > max_area and len(approx) == 4:
#                 biggest = approx
#                 max_area = area
#     return biggest


# # Open a connection to the webcam (use 0 for the default camera)
# cap = cv2.VideoCapture(0)

# while True:
#     # Capture frame-by-frame
#     ret, frame = cap.read()
#     img_original = frame.copy()
    
#     # Break the loop if there's an issue capturing the frame
#     if not ret:
#         print("Error: Failed to capture frame.")
#         break

#     # Grayscale
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Find Canny edges
#     edged = cv2.Canny(gray, 30, 200)

#     # Finding Contours
#     contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

#     # Print the number of contours found
#     print("Number of Contours found = " + str(len(contours)))

#     biggest = biggest_contour(contours)
#     print("Number of Biggest Contours found = " + str(len(biggest)))
#     if len(biggest !=0):

#         cv2.drawContours(frame, [biggest], -1, (0, 255, 0), 3)
#         # Pixel values in the original image
#         points = biggest.reshape(4, 2)
#         input_points = np.zeros((4, 2), dtype="float32")

#         points_sum = points.sum(axis=1)
#         input_points[0] = points[np.argmin(points_sum)]
#         input_points[3] = points[np.argmax(points_sum)]

#         points_diff = np.diff(points, axis=1)
#         input_points[1] = points[np.argmin(points_diff)]
#         input_points[2] = points[np.argmax(points_diff)]

#         (top_left, top_right, bottom_right, bottom_left) = input_points
#         bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + ((bottom_right[1] - bottom_left[1]) ** 2))
#         top_width = np.sqrt(((top_right[0] - top_left[0]) ** 2) + ((top_right[1] - top_left[1]) ** 2))
#         right_height = np.sqrt(((top_right[0] - bottom_right[0]) ** 2) + ((top_right[1] - bottom_right[1]) ** 2))
#         left_height = np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))

#         # Output image size
#         max_width = max(int(bottom_width), int(top_width))
#         # max_height = max(int(right_height), int(left_height))
#         max_height = int(max_width * 1.414)  # for A4

#         # Desired points values in the output image
#         converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])

#         # Perspective transformation
#         matrix = cv2.getPerspectiveTransform(input_points, converted_points)
#         img_output = cv2.warpPerspective(img_original, matrix, (max_width, max_height))

#         # Resize the image to the fixed size
#         img_output = cv2.resize(img_output, (460, 800))

#         # Image shape modification for hstack
#         gray = np.stack((gray,) * 3, axis=-1)
#         edged = np.stack((edged,) * 3, axis=-1)

#         img_hor = np.hstack((img_original, gray, edged))
#         cv2.imshow("Contour detection", img_hor)
#         cv2.imshow("Warped perspective", img_output)


#     # Display the frame with contours
#     cv2.imshow('Contours', frame)

#     # Break the loop if 'q' key is pressed
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the webcam and close all windows
# cap.release()
# cv2.destroyAllWindows()

In [3]:
from PIL import Image
import cv2
import numpy as np
import pytesseract
import platform
if platform.system() == "Windows":
    pytesseract.pytesseract.tesseract_cmd = r'C:\Users\B92383\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
custom_config = r'--oem 2 --psm 12 -l fra+eng'

def image2text(image):
    text = ''
    

    text = pytesseract.image_to_string(image, config=custom_config)

    if len(text)!=0:
        return text
    else:
        return "no text detected"

def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 1000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.015 * peri, True)
            if area > max_area and len(approx) == 4:
                biggest = approx
                max_area = area
    return biggest


img = cv2.imread('carte-grise-specimen_rot.jpg')
img = np.array(Image.fromarray(img.copy()).rotate(65,resample=Image.BICUBIC, expand=True))
img_original = img.copy()

# Image modification
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 20, 30, 30)
edged = cv2.Canny(gray, 10, 20)

# Contour detection
contours, hierarchy = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(hierarchy)

contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

biggest = biggest_contour(contours)

cv2.drawContours(img, [biggest], -1, (0, 255, 0), 3)

# Pixel values in the original image
points = biggest.reshape(4, 2)
input_points = np.zeros((4, 2), dtype="float32")

points_sum = points.sum(axis=1)
input_points[0] = points[np.argmin(points_sum)]
input_points[3] = points[np.argmax(points_sum)]

points_diff = np.diff(points, axis=1)
input_points[1] = points[np.argmin(points_diff)]
input_points[2] = points[np.argmax(points_diff)]

(top_left, top_right, bottom_right, bottom_left) = input_points
bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + ((bottom_right[1] - bottom_left[1]) ** 2))
top_width = np.sqrt(((top_right[0] - top_left[0]) ** 2) + ((top_right[1] - top_left[1]) ** 2))
right_height = np.sqrt(((top_right[0] - bottom_right[0]) ** 2) + ((top_right[1] - bottom_right[1]) ** 2))
left_height = np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))

# Output image size
max_width = max(int(bottom_width), int(top_width))
# max_height = max(int(right_height), int(left_height))
max_height = int(max_width * 2.035)  # for A4

converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])

# Perspective transformation
matrix = cv2.getPerspectiveTransform(input_points, converted_points)
img_output = cv2.warpPerspective(img_original, matrix, (max_width, max_height))

# for rot in range(4):
            
#             img_output_to_test_text = img_output[0:30, :]
#             cv2.imshow("Warped perspective", img_output_to_test_text)
#             cv2.waitKey(0)
            
#             word_found_to_test = image2text(img_output_to_test_text)
            
#             print(word_found_to_test)
#             if "immatriculation" in word_found_to_test.lower():
#                 print("OK")
#                 break
#             else:
#                 img_output = np.array(Image.fromarray(img_output.copy()).rotate(90,resample=Image.BICUBIC, expand=True))
 # Resize the image to the fixed size
# img_output = cv2.resize(img_output, (460, 800),interpolation=cv2.INTER_CUBIC)

# Image shape modification for hstack
gray = np.stack((gray,) * 3, axis=-1)
edged = np.stack((edged,) * 3, axis=-1)

img_hor = np.hstack((img_original, gray, edged, img))
cv2.imshow("Contour detection", img_hor)
cv2.imshow("Warped perspective", img_output)

# cv2.imwrite('output/document.jpg', img_output)

cv2.waitKey(0)
cv2.destroyAllWindows()


[[[   1   -1   -1   -1]
  [   2    0   -1   -1]
  [1748    1    3   -1]
  ...
  [  -1 1746   -1    2]
  [1749    2   -1   -1]
  [  -1 1748   -1   -1]]]


In [4]:
def order_points(pts):
    '''Rearrange coordinates to order:
       top-left, top-right, bottom-right, bottom-left'''
    rect = np.zeros((4, 2), dtype='float32')
    pts = np.array(pts)
    s = pts.sum(axis=1)
    # Top-left point will have the smallest sum.
    rect[0] = pts[np.argmin(s)]
    # Bottom-right point will have the largest sum.
    rect[2] = pts[np.argmax(s)]

    diff = np.diff(pts, axis=1)
    # Top-right point will have the smallest difference.
    rect[1] = pts[np.argmin(diff)]
    # Bottom-left will have the largest difference.
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect.astype('int').tolist()


def find_dest(pts):
    (tl, tr, br, bl) = pts
    # Finding the maximum width.
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # Finding the maximum height.
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # Final destination co-ordinates.
    destination_corners = [[0, 0], [maxWidth, 0], [maxWidth, maxHeight], [0, maxHeight]]

    return order_points(destination_corners)



def scan(img):
    # Resize image to workable size
    # dim_limit = 1080
    # max_dim = max(img.shape)
    # if max_dim > dim_limit:
    #     resize_scale = dim_limit / max_dim
    #     img = cv2.resize(img, None, fx=resize_scale, fy=resize_scale)
    
    # img = cv2.resize(img, (1400, 1800))
    # Create a copy of resized original image for later use

    orig_img = img.copy()
    # Repeated Closing operation to remove text from the document.
    kernel = np.ones((5, 5), np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel, iterations=3)
    # GrabCut
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1, 65), np.float64)
    fgdModel = np.zeros((1, 65), np.float64)
    rect = (20, 20, img.shape[1] - 20, img.shape[0] - 20)
    cv2.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    img = img * mask2[:, :, np.newaxis]

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (11, 11), 0)
    # Edge Detection.
    canny = cv2.Canny(gray, 0, 200)
    canny = cv2.dilate(canny, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Finding contours for the detected edges.
    contours, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Keeping only the largest detected contour.
    page = sorted(contours, key=cv2.contourArea, reverse=True)[:5]

    # Detecting Edges through Contour approximation.
    # Loop over the contours.
    if len(page) == 0:
        return orig_img
    for c in page:
        # Approximate the contour.
        epsilon = 0.02 * cv2.arcLength(c, True)
        corners = cv2.approxPolyDP(c, epsilon, True)
        # If our approximated contour has four points.
        if len(corners) == 4:
            break
    # Sorting the corners and converting them to desired shape.
    corners = sorted(np.concatenate(corners).tolist())
    # For 4 corner points being detected.
    corners = order_points(corners)

    destination_corners = find_dest(corners)

    # Getting the homography.
    M = cv2.getPerspectiveTransform(np.float32(corners), np.float32(destination_corners))
    # Perspective transform using homography.
    final = cv2.warpPerspective(orig_img, M, (destination_corners[2][0], destination_corners[2][1]),
                                flags=cv2.INTER_LINEAR)
    final = cv2.resize(final, (1200, 1600))
    return final

In [5]:
import cv2
import numpy as np


img = cv2.imread('ci.jpg')

img_original = img.copy()

img_output = scan(img)
img_output = cv2.resize(img_output, (600,800))

cv2.imshow("Contour detection", img_original)
cv2.imshow("Warped perspective", img_output)

cv2.imwrite('output/document.jpg', img_output)
cv2.waitKey(0)
cv2.destroyAllWindows()